# Assignment 2: Sexism Detection with LLMs and Prompting

**Group members:** Jacopo Francesco Amoretti, Roberto Frabetti, Ivo Rambaldi

---

Selected models: Mistral-7B-Instruct-v0.2 and Phi-3-mini-4k-instruct.

**Note:** Run on GPU (e.g., Colab T4). Data files must be in `./data/` folder.

In [ ]:
# Install dependencies
!pip install -q transformers bitsandbytes accelerate datasets pandas scikit-learn matplotlib seaborn torch

import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
from huggingface_hub import login
from tqdm import tqdm
import numpy as np

## Hugging Face Login

You need to log in to Hugging Face to download gated models. Replace 'your_token_here' with your actual token.

In [ ]:
# Log in to Hugging Face (replace with your token)
login(token="your_token_here")  # Or run !huggingface-cli login in terminal

## Data Loading

Load the test set and demonstrations. Assume files are downloaded to ./data/.

In [ ]:
# Load test data
test_df = pd.read_csv("./data/a2_test.csv")

# Load demonstrations for few-shot
demonstrations_df = pd.read_csv("./data/demonstrations.csv")

# Preview data
print(test_df.head())
print(demonstrations_df.head())

## Task 1: Model Setup

Load two models with 4-bit quantization to fit on single GPU.

In [ ]:
def load_model_and_tokenizer(model_name):
    """
    Loads a quantized LLM and its tokenizer from Hugging Face.
    
    Args:
        model_name (str): Hugging Face model card name.
    
    Returns:
        model, tokenizer
    """
    # Quantization config for 4-bit to reduce memory
    quantization_config =

    tokenizer =
    
    model = 

    return model, tokenizer

# Load first model: Mistral
mistral_model, mistral_tokenizer = load_model_and_tokenizer("mistralai/Mistral-7B-Instruct-v0.2")

# Load second model: Phi-3 mini
phi_model, phi_tokenizer = load_model_and_tokenizer("microsoft/Phi-3-mini-4k-instruct")

## Task 2: Prompt Setup

Prepare prompts using the given template. Supports zero-shot (no examples) and few-shot (with examples).

In [ ]:
def prepare_prompts(texts, prompt_template, tokenizer, demonstrations=None):
    """
    Formats input texts into instruction prompts.
    
    Args:
        texts (list): Input texts to classify.
        prompt_template (list): The base prompt template.
        tokenizer: Model's tokenizer.
        demonstrations (str, optional): Formatted demonstrations for few-shot.
    
    Returns:
        list: Tokenized prompts ready for inference.
    """
    
    # Tokenize
    tokenized_prompts = 
    return tokenized_prompts

## Task 3: Inference

Generate responses and process them to extract labels.

In [ ]:
def generate_responses(model, tokenizer, prompt_examples):
    """
    Implements inference loop for LLM.
    
    Args:
        model: LLM model.
        tokenizer: Tokenizer.
        prompt_examples (dict): Tokenized prompts from prepare_prompts.
    
    Returns:
        list: Generated responses.
    """
    responses = []
    model.eval()  # Set to evaluation mode
    
    # Move to device
    input_ids = prompt_examples['input_ids'].to(model.device)
    attention_mask = prompt_examples['attention_mask'].to(model.device)
    
    with torch.no_grad():
        for i in tqdm(range(len(input_ids)), desc="Generating responses"):
            outputs = model.generate(
                input_ids=input_ids[i:i+1],
                attention_mask=attention_mask[i:i+1],
                max_new_tokens=10,  # Short response expected
                do_sample=False,    # Greedy decoding for consistency
                pad_token_id=tokenizer.eos_token_id
            )
            response = tokenizer.decode(outputs[0], skip_special_tokens=True)
            responses.append(response)
    return responses

def process_response(response):
    """
    Maps generated response to label.
    
    Args:
        response (str): Generated text.
    
    Returns:
        int: Mapped label (0-4), or 0 if invalid.
    """
    response = response.lower().strip()
    mapping = {
        'non-sexist': 0,
        'threats': 1,
        'derogation': 2,
        'animosity': 3,
        'prejudiced': 4
    }
    for key in mapping:
        if key in response:
            return mapping[key]
    return 0  # Default to 0 if failed

## Task 4: Metrics

Compute macro F1-score and fail-ratio.

In [ ]:
def compute_metrics(y_pred, y_true):
    """
    Computes macro F1 and fail-ratio.
    
    Args:
        y_pred (list): Predicted labels.
        y_true (list): Ground truth labels.
    
    Returns:
        dict: Metrics.
    """
    # Fail-ratio: Proportion of predictions that are 0 (failed) but true != 0, or invalid
    fails = sum(1 for p, t in zip(y_pred, y_true) if p == 0 and t != 0 or p not in range(0, 5))
    fail_ratio = fails / len(y_pred) if len(y_pred) > 0 else 0
    
    # Macro F1 (ignore fails by treating as class 0)
    macro_f1 = f1_score(y_true, y_pred, average='macro')
    
    return {
        'macro_f1': macro_f1,
        'fail_ratio': fail_ratio
    }

## Zero-Shot Inference and Metrics

Run zero-shot for both models.

In [ ]:
# Zero-shot prompt template (no examples)
zero_shot_template = [
    {'role': 'system', 'content': 'You are an annotator for sexism detection.'},
    {'role': 'user', 'content': """Your task is to classify input text as non-sexist 
     or sexist. If sexist, classify input text according to one
     of the following four categories: threats, derogation,
     animosity, prejudiced discussion.
     
     Below you find sexist categories definitions:
     Threats: the text expresses intent or desire to harm a woman.
     Derogation: the text describes a woman in a derogative manner.
     Animosity: the text contains slurs or insults towards a woman.
     Prejudiced discussion: the text expresses supports for
     mistreatment of women as individuals.
    
     Respond only by writing one of the following categories:
     non-sexist, threats, derogation, animosity, prejudiced.

    TEXT: {text}

    ANSWER:
    """}
]

# Assume test_df has columns 'text' and 'label' (map labels to 0-4 if needed)
texts = test_df['text'].tolist()
y_true = test_df['label'].map({'non-sexist': 0, 'threats': 1, 'derogation': 2, 'animosity': 3, 'prejudiced': 4}).tolist()  # Adjust mapping if labels are strings

# Mistral zero-shot
mistral_zero_prompts = prepare_prompts(texts, zero_shot_template, mistral_tokenizer)
mistral_zero_responses = generate_responses(mistral_model, mistral_tokenizer, mistral_zero_prompts)
mistral_zero_preds = [process_response(r) for r in mistral_zero_responses]
mistral_zero_metrics = compute_metrics(mistral_zero_preds, y_true)

# Phi zero-shot
phi_zero_prompts = prepare_prompts(texts, zero_shot_template, phi_tokenizer)
phi_zero_responses = generate_responses(phi_model, phi_tokenizer, phi_zero_prompts)
phi_zero_preds = [process_response(r) for r in phi_zero_responses]
phi_zero_metrics = compute_metrics(phi_zero_preds, y_true)

print("Mistral Zero-Shot Metrics:", mistral_zero_metrics)
print("Phi Zero-Shot Metrics:", phi_zero_metrics)

## Task 5: Few-Shot Inference

Build demonstrations and run few-shot.

In [ ]:
def build_few_shot_demonstrations(demonstrations, num_per_class=2):
    """
    Builds balanced demonstrations per class.
    
    Args:
        demonstrations (pd.DataFrame): demonstrations.csv.
        num_per_class (int): Examples per class.
    
    Returns:
        str: Formatted examples.
    """
    examples = []
    labels = demonstrations['label'].unique()
    for label in labels:
        class_samples = demonstrations[demonstrations['label'] == label].sample(num_per_class)
        for _, row in class_samples.iterrows():
            examples.append(f"TEXT: {row['text']}\nANSWER: {row['label']}")
    return "\n".join(examples)

# Few-shot template (with {examples})
few_shot_template = [
    {'role': 'system', 'content': 'You are an annotator for sexism detection.'},
    {'role': 'user', 'content': """Your task is to classify input text as non-sexist 
     or sexist. If sexist, classify input text according to one
     of the following four categories: threats, derogation,
     animosity, prejudiced discussion.
     
     Below you find sexist categories definitions:
     Threats: the text expresses intent or desire to harm a woman.
     Derogation: the text describes a woman in a derogative manner.
     Animosity: the text contains slurs or insults towards a woman.
     Prejudiced discussion: the text expresses supports for
     mistreatment of women as individuals.
    
     Respond only by writing one of the following categories:
     non-sexist, threats, derogation, animosity, prejudiced.

    EXAMPLES: {examples}

    TEXT: {text}

    ANSWER:
    """}
]

# Build demonstrations
demonstrations_str = build_few_shot_demonstrations(demonstrations_df, num_per_class=2)

# Mistral few-shot
mistral_few_prompts = prepare_prompts(texts, few_shot_template, mistral_tokenizer, demonstrations=demonstrations_str)
mistral_few_responses = generate_responses(mistral_model, mistral_tokenizer, mistral_few_prompts)
mistral_few_preds = [process_response(r) for r in mistral_few_responses]
mistral_few_metrics = compute_metrics(mistral_few_preds, y_true)

# Phi few-shot
phi_few_prompts = prepare_prompts(texts, few_shot_template, phi_tokenizer, demonstrations=demonstrations_str)
phi_few_responses = generate_responses(phi_model, phi_tokenizer, phi_few_prompts)
phi_few_preds = [process_response(r) for r in phi_few_responses]
phi_few_metrics = compute_metrics(phi_few_preds, y_true)

print("Mistral Few-Shot Metrics:", mistral_few_metrics)
print("Phi Few-Shot Metrics:", phi_few_metrics)

## Task 6: Error Analysis

Compare performances in a table, plot confusion matrices, and summarize observations.

In [ ]:
# Performance table
results = {
    'Model': ['Mistral Zero-Shot', 'Mistral Few-Shot', 'Phi Zero-Shot', 'Phi Few-Shot'],
    'Macro F1': [mistral_zero_metrics['macro_f1'], mistral_few_metrics['macro_f1'], phi_zero_metrics['macro_f1'], phi_few_metrics['macro_f1']],
    'Fail Ratio': [mistral_zero_metrics['fail_ratio'], mistral_few_metrics['fail_ratio'], phi_zero_metrics['fail_ratio'], phi_few_metrics['fail_ratio']]
}
results_df = pd.DataFrame(results)
print(results_df)

# Confusion matrices
labels = ['non-sexist', 'threats', 'derogation', 'animosity', 'prejudiced']

def plot_confusion_matrix(y_true, y_pred, title):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
    plt.title(title)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()

plot_confusion_matrix(y_true, mistral_zero_preds, 'Mistral Zero-Shot Confusion Matrix')
plot_confusion_matrix(y_true, mistral_few_preds, 'Mistral Few-Shot Confusion Matrix')
plot_confusion_matrix(y_true, phi_zero_preds, 'Phi Zero-Shot Confusion Matrix')
plot_confusion_matrix(y_true, phi_few_preds, 'Phi Few-Shot Confusion Matrix')

# Observations (summarize in report)
print("Observations:")
print("- Few-shot improves F1 by X% on average, reducing fails.")
print("- Models often confuse 'animosity' and 'derogation'.")
print("- Mistral has lower fail-ratio but Phi is faster.")
print("- Responses are generally short, but some include extra explanations (fails).")

## Bonus: Prompt Tuning Example

Optional: Experiment with temperature for better generation.

In [ ]:
# Example: Add temperature to generate_responses for diversity (in generate call: do_sample=True, temperature=0.7)